# CELLULAR AUTOMATA (2d)

Non penso debba dire nulla, se non sai cosa siano i CA vai su youtube e 2 minuti l'hai capito.

Unica cosa: usiamo le regole in formato **ALIVE / BORN / STATE / NEIGHBOURHOOD**

In [134]:
import pygame 
import numpy as np
import random
import copy
import time

Definiamo per il momento il "moore neighbourhood", che è semplicemente il quadrato che accerchia la nostra cellula (nel disegno le caselle con la M sono il vicinato che consideriamo, noi siamo l'emoji contenta). La funzione restituisce direttamente il numero di celle vive.

|M |M |M| 
|-|-|-|
|**M** |:) |**M** |
|**M** |**M** |**M** |

In [135]:
def moore_neighbourhood(grid, index):
    y = index[0]
    x = index[1]

    side = len(grid)
    moore = []
    for i in range(y-1, y+2):
        k = i%side
        row = []
        for j in range (x-1, x + 2):
            w = j%side
            row.append(grid[k, w])
        moore.append(row)
    
    moore = np.array(moore)
    offset = 1 if grid[index] !=0 else 0 # non dobbiamo considerare la cellula stessa
    return np.count_nonzero(moore) - offset 

Questo invece è il "von neighbourhood" (o così mi pare si chiami). Prevede solo gli elementi sopra e a lato. 

| |M | | 
|-|-|-|
|**M** |:) |**M** |
| |**M** | |

In [136]:
def von_neighbourhood(grid, index):
    y = index[0]
    x = index[1]

    side = len(grid)
    von = []
    
    for i in range (y-1, y+2):
        k = i%side
        if y != k:
            von.append(grid[k, x])
    
    for j in range (x-1, x+2):
        w = j%side
        if x != w:
            von.append(grid[y, w])
    
    von = np.array(von)
    return np.count_nonzero(von)

Implementiamo anche il "Diagonal Neighbourhood" che è semplicemente l'esclusione del von dal moore.

|M |  |M| 
|-|-|-|
| |:) | |
|**M** | |**M** |

In [137]:
def diag_neighbourhood(grid, index):
    return moore_neighbourhood(grid, index) - von_neighbourhood(grid, index)

In [138]:
g = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])
print(g)
von_neighbourhood(g, (2, 1))


[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


4

Definiamo una classe per le regole. Affidiamo alla regola l'update della cella e dell'intera griglia.

In [149]:
class Rule:
    
    def __init__(self, alive, born, state, neighbourhood):
        self.alive = alive
        self.born = born
        self.state = state
        self.neighbourhood = neighbourhood

    def update_cell(self, grid, index):
        pass

    def update_grid(self, grid):
        pass

In [155]:
class CArule(Rule):

    def __init__(self, alive, born, state, neighbourhood):
        super().__init__(alive, born, state, neighbourhood)

    def update_cell(self, grid, index):
        if grid[index] == 1:
            if self.neighbourhood(grid, index) in self.alive:
                return 1
            else:
                return -self.state
        elif grid[index] == 0:
            if self.neighbourhood(grid, index) in self.born:
                return 1
            else:
                return 0
        else:
            return grid[index] + 1
        
    def update_grid(self, grid):
        side = len(grid)
        new_grid = np.zeros_like(grid)
        for i in range (side):
            for j in range(side):
                new_grid[i, j] = self.update_cell(grid, (i, j))
        return new_grid

Qui definiamo le due cose principali. 

1. la **grid**, cioè l'ambiente (quadrato) in cui la CA evolve
2. la **rule**, cioè la regola che devono seguire le singole cellule

In [156]:
side = 100
grid = np.zeros(shape=(side, side))

In [157]:
rule = CArule([1, 3], [2, 4], 5, moore_neighbourhood)

Questa è l'interfaccia per selezionare le cellule iniziali.

In [158]:
pygame.init()

window_size = 600
cell_dimension = window_size//side
background_color = (0, 0, 0)
line_color = (255, 255, 255)

fps = 60

screen = pygame.display.set_mode((window_size, window_size))
pygame.display.set_caption("Cellular Automata - Setup")

clock = pygame.time.Clock()

while True:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos[0]//cell_dimension, event.pos[1]//cell_dimension
            if 0 <= x < side and 0 <= y < side:
                grid[y, x] = 0 if grid[y, x] == 1 else 1

    screen.fill(background_color)

    for row in range(side):
        for column in range(side):
            x = column * cell_dimension
            y = row * cell_dimension
            cell_color = (0, 0, 0) if grid[row, column] == 0 else (255, 255, 255)
            pygame.draw.rect(screen, cell_color, (x, y, cell_dimension, cell_dimension)) # questo è per la cella attuale
            #pygame.draw.rect(screen, line_color, (x, y, cell_dimension, cell_dimension), 1) # questo è per i bordi

    pygame.display.flip()

    clock.tick(fps)

error: display Surface quit

E' questa è l'attuale interfaccia.

In [163]:
pygame.init()

window_size = 600
cell_dimension = window_size//side
line_color = (255, 255, 255)

light_theme = False
background_color = (255, 255, 255) if light_theme else (0, 0, 0)

fps = 30

screen = pygame.display.set_mode((window_size, window_size))
pygame.display.set_caption("Cellular Automata")

clock = pygame.time.Clock()

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()

    for row in range(side):
        for column in range(side):
            x = column * cell_dimension
            y = row * cell_dimension
            k = grid[row, column]
            
            # light theme
            if light_theme:
                shade = 255 if k == 1 else 255*(((rule.state + k + 1)/rule.state))
                cell_color = (255, 255, 255) if k == 0 else (shade, shade, 255)
            # dark theme
            else:
                shade = 255 if k == 1 else 255*(1 - ((rule.state + k + 1)/rule.state))
                cell_color = (0, 0, 0) if k == 0 else (0, 0, shade)

            pygame.draw.rect(screen, cell_color, (x, y, cell_dimension, cell_dimension)) # questo è per la cella attuale
            #pygame.draw.rect(screen, line_color, (x, y, cell_dimension, cell_dimension), 1) # questo è per i bordi
    
    grid = rule.update_grid(grid)

    pygame.display.flip()

    clock.tick(fps)


error: display Surface quit